In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import os

In [3]:
file_path = os.listdir('cleaned_data')
file_path

['农业.csv',
 '黑色金属矿采选业.csv',
 '畜牧业.csv',
 '煤炭开采和洗选业.csv',
 '农、林、牧、渔服务业.csv',
 '林业.csv',
 '农副食品加工业.csv',
 '石油和天然气开采业.csv',
 '酒、饮料和精制茶制造业.csv',
 '食品制造业.csv',
 '有色金属矿采选业.csv',
 '非金属矿采选业.csv',
 '开采辅助活动.csv',
 '渔业.csv']

In [10]:
whole = []
for i in range(len(file_path)):
    df = pd.read_csv('cleaned_data/'+file_path[i],index_col=0)
    whole.append(df)
whole1 = pd.concat(whole)
whole1
whole1.tocsv = ('cleaned_data.csv')

In [2]:
    
raw_data = pd.read_csv('cleaned_data.csv',index_col=0)
one_month = raw_data[raw_data['year']==year]
pivot_m = pd.pivot_table(data=one_month,values='return',columns='简称',index='日期')
corr=pivot_m.corr()
distance_m = pd.DataFrame(np.tril(np.sqrt(2*(1-corr))))

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_data.csv'